# Bloque 1 · ¿Qué es Retrieval-Augmented Generation (RAG)?

Este bloque introduce los conceptos esenciales de un flujo RAG y cómo cada componente contribuye a obtener respuestas mejor informadas a partir de fuentes propias.


## ¿Por qué RAG?

Los modelos de lenguaje grandes (LLM) poseen conocimiento general, pero no siempre están actualizados ni conocen los detalles internos de una organización. Retrieval-Augmented Generation (RAG) permite enriquecer las respuestas de un modelo con información específica proveniente de una base de conocimiento curada.

En otras palabras, RAG combina dos mundos:
- **Recuperación**: localizar fragmentos relevantes en documentos propios.
- **Generación**: redactar una respuesta usando tanto la pregunta como los fragmentos recuperados.


## Componentes de un sistema RAG

1. **Fuentes de conocimiento**: documentos, páginas web, manuales o bases de datos que contienen la información confiable.
2. **Ingesta y limpieza**: procesos que normalizan, eliminan ruido y preparan el texto.
3. **Chunking**: división en fragmentos manejables (chunks) que capturen ideas completas.
4. **Modelos de embeddings**: convierten cada fragmento en un vector numérico que preserva significado.
5. **Base vectorial**: almacena los vectores y permite realizar búsquedas por similitud.
6. **Recuperador**: dada una consulta, encuentra los vectores más cercanos.
7. **Generador**: redacta la respuesta final usando la consulta y los fragmentos recuperados.


## Flujo típico de extremo a extremo

1. Se seleccionan y preparan las fuentes confiables.
2. Se crean fragmentos y sus embeddings.
3. Se almacenan en una base vectorial.
4. Una pregunta del usuario se transforma en vector.
5. Se recuperan los fragmentos más similares.
6. Un modelo genera una respuesta apoyándose en esos fragmentos.


## Actividad práctica: simulación de vector search

Trabajaremos con un ejemplo muy simple usando vectores predefinidos que preservan relaciones semánticas. En sistemas reales, estos vectores se generan mediante modelos de transformers entrenados que capturan el significado de las palabras, pero aquí usaremos vectores ya creados para entender el concepto sin ejecutar ningún modelo.


### Paso 1. Definir una base de conocimiento simple

Trabajaremos con un corpus muy básico de solo 3 documentos cortos para entender el concepto.


In [2]:
corpus = [
    {
        "id": "doc1",
        "contenido": "Los gatos son mascotas populares."
    },
    {
        "id": "doc2",
        "contenido": "Los perros también son mascotas comunes."
    },
    {
        "id": "doc3",
        "contenido": "La programación requiere práctica constante."
    },
]

print(f"Documentos disponibles: {len(corpus)}")
for doc in corpus:
    print(f"  - {doc['id']}: {doc['contenido']}")


Documentos disponibles: 3
  - doc1: Los gatos son mascotas populares.
  - doc2: Los perros también son mascotas comunes.
  - doc3: La programación requiere práctica constante.


### Paso 2. Fragmentar y tokenizar

Cada documento ya es una frase completa, así que simplemente lo dividiremos en palabras (tokens).


In [10]:
def tokenize(text):
    """Convierte el texto en una lista de tokens (palabras) limpias, incluyendo signos de pregunta como tokens separados."""
    # Reemplazar signos de puntuación con espacios para que se separen durante el split
    cleaned = text.lower().replace(".", "").replace(",", "").replace("?", " ? ").replace("¿", "¿ ")
    return cleaned.split()

chunks = []
for doc in corpus:
    tokens = tokenize(doc["contenido"])
    chunks.append({
        "doc_id": doc["id"],
        "fragmento": doc["contenido"],
        "tokens": tokens,
    })

print(f"Chunks generados: {len(chunks)}\n")
for chunk in chunks:
    print(f"{chunk['doc_id']}: {chunk['fragmento']}")
    print(f"  Tokens: {chunk['tokens']}\n")

Chunks generados: 3

doc1: Los gatos son mascotas populares.
  Tokens: ['los', 'gatos', 'son', 'mascotas', 'populares']

doc2: Los perros también son mascotas comunes.
  Tokens: ['los', 'perros', 'también', 'son', 'mascotas', 'comunes']

doc3: La programación requiere práctica constante.
  Tokens: ['la', 'programación', 'requiere', 'práctica', 'constante']



### Paso 3. Embeddings predefinidos

En sistemas reales, los modelos de transformers (como BERT, GPT, etc.) generan vectores que capturan el significado de las palabras. Palabras con significados similares tienen vectores cercanos en el espacio.

Aquí usaremos vectores predefinidos que ya preservan estas relaciones semánticas. Por ejemplo:
- "gatos" y "perros" (ambas son mascotas) tendrán vectores similares
- "mascotas" y "animales" tendrán vectores cercanos
- "programación" tendrá un vector diferente (tema distinto)


In [4]:
# Embeddings predefinidos que preservan relaciones semánticas
# En la práctica, estos vectores serían generados por un modelo transformer entrenado

# Ahora usamos vectores de dimensión 5 para permitir una mejor diferenciación.
# Dimensión 1: Animal general / Ser vivo
# Dimensión 2: Característica de mascota
# Dimensión 3: Característica específica de GATO
# Dimensión 4: Característica específica de PERRO
# Dimensión 5: Característica específica de Programación
import numpy as np

embeddings_dict = {
    "los": np.array([0.1, 0.0, 0.0, 0.0, 0.0]),
    "gatos": np.array([0.8, 0.7, 0.9, 0.1, 0.0]),      # Animal, Mascota, Gato-específico
    "son": np.array([0.0, 0.1, 0.0, 0.0, 0.0]),
    "mascotas": np.array([0.9, 0.8, 0.3, 0.3, 0.0]),  # Animal, Mascota, neutral en especie
    "populares": np.array([0.2, 0.3, 0.1, 0.1, 0.0]),
    "perros": np.array([0.8, 0.7, 0.1, 0.9, 0.0]),  # Animal, Mascota, Perro-específico
    "también": np.array([0.0, 0.1, 0.0, 0.0, 0.0]),
    "comunes": np.array([0.2, 0.3, 0.1, 0.1, 0.0]),
    "la": np.array([0.1, 0.0, 0.0, 0.0, 0.0]),
    "programación": np.array([0.1, 0.0, 0.0, 0.0, 0.9]),  # Tema diferente, Programación-específico
    "requiere": np.array([0.0, 0.1, 0.0, 0.0, 0.3]),
    "práctica": np.array([0.0, 0.2, 0.0, 0.0, 0.6]),
    "constante": np.array([0.0, 0.1, 0.0, 0.0, 0.4]),
}

# Normalizar todos los vectores
for palabra in embeddings_dict:
    norm = np.linalg.norm(embeddings_dict[palabra])
    if norm > 0:
        embeddings_dict[palabra] = embeddings_dict[palabra] / norm

print("Embeddings predefinidos (vectores normalizados):")
print("\nPalabras relacionadas con mascotas:")
print(f"  'gatos':    {embeddings_dict['gatos']}")
print(f"  'perros':   {embeddings_dict['perros']}")
print(f"  'mascotas': {embeddings_dict['mascotas']}")

print("\nPalabras relacionadas con programación:")
print(f"  'programación': {embeddings_dict['programación']}")
print(f"  'práctica':     {embeddings_dict['práctica']}")

print("\nNota: Los vectores de 'gatos' y 'perros' ahora tienen valores más distintivos en sus dimensiones específicas (Dim3 para gatos, Dim4 para perros) aunque mantienen similitud en las dimensiones de 'animal' y 'mascota'.")
print("      'programación' sigue siendo un tema diferente con alta carga en Dim5.")

Embeddings predefinidos (vectores normalizados):

Palabras relacionadas con mascotas:
  'gatos':    [0.5728919  0.50128041 0.64450339 0.07161149 0.        ]
  'perros':   [0.5728919  0.50128041 0.07161149 0.64450339 0.        ]
  'mascotas': [0.7049344  0.62660836 0.23497813 0.23497813 0.        ]

Palabras relacionadas con programación:
  'programación': [0.11043153 0.         0.         0.         0.99388373]
  'práctica':     [0.         0.31622777 0.         0.         0.9486833 ]

Nota: Los vectores de 'gatos' y 'perros' ahora tienen valores más distintivos en sus dimensiones específicas (Dim3 para gatos, Dim4 para perros) aunque mantienen similitud en las dimensiones de 'animal' y 'mascota'.
      'programación' sigue siendo un tema diferente con alta carga en Dim5.


### Paso 4. Convertir chunks en vectores

Para representar un fragmento completo (que tiene múltiples palabras), calcularemos el promedio de los vectores de todas sus palabras y luego lo normalizamos. Esto nos da un único vector que representa el significado del fragmento completo.


In [5]:
def chunk_to_vector(tokens, embeddings_dict):
    """
    Convierte una lista de tokens en un vector promediando los embeddings de cada palabra.
    Si una palabra no está en el diccionario, se omite.
    """
    # Asumiendo que todos los embeddings en embeddings_dict tienen la misma dimensión.
    # Si no hay tokens, o si el diccionario está vacío, devuelve un vector de ceros de la dimensión correcta.
    if not tokens or not embeddings_dict:
        # Determina la dimensión a partir de un embedding existente o usa un valor predeterminado (e.g., 5)
        if embeddings_dict:
            sample_vector = next(iter(embeddings_dict.values()))
            return np.zeros(len(sample_vector))
        else:
            return np.zeros(5) # Dimensión por defecto si no hay embeddings

    vectors = []
    for token in tokens:
        if token in embeddings_dict:
            vectors.append(embeddings_dict[token])

    if not vectors:
        # Si no se encontraron palabras en el diccionario, devuelve un vector de ceros
        sample_vector = next(iter(embeddings_dict.values()))
        return np.zeros(len(sample_vector))

    # Promedio de los vectores de palabras
    vector_promedio = np.mean(vectors, axis=0)

    # Normalizar el vector resultante
    norm = np.linalg.norm(vector_promedio)
    if norm > 0:
        vector_promedio = vector_promedio / norm

    return vector_promedio

# Convertir todos los chunks a vectores
chunk_vectors = []
for chunk in chunks:
    vector = chunk_to_vector(chunk["tokens"], embeddings_dict)
    chunk_vectors.append(vector)
    print(f"{chunk['doc_id']}: {chunk['fragmento']}")
    print(f"  Vector: {vector}\n")

chunk_vectors = np.array(chunk_vectors)
print(f"Total de vectores: {chunk_vectors.shape}")

doc1: Los gatos son mascotas populares.
  Vector: [0.66145139 0.68707912 0.26931279 0.13369728 0.        ]

doc2: Los perros también son mascotas comunes.
  Vector: [0.56279391 0.7860125  0.11375592 0.22914397 0.        ]

doc3: La programación requiere práctica constante.
  Vector: [0.27004276 0.21278667 0.         0.         0.9390414 ]

Total de vectores: (3, 5)


### Paso 5. Recuperar fragmentos por similitud

Compararemos la representación vectorial de la pregunta con cada chunk usando similitud coseno. Esta métrica mide el ángulo entre dos vectores: valores cercanos a 1 indican alta similitud, mientras que valores cercanos a 0 indican baja similitud.


In [6]:
def cosine_similarity(vector_a, vector_b):
    """Calcula la similitud coseno entre dos vectores."""
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    if denom == 0:
        # Si uno de los vectores es cero, la similitud es 0.0 para evitar división por cero.
        return 0.0
    return float(np.dot(vector_a, vector_b) / denom)

def retrieve(query, top_k=3):
    """
    Recupera los top_k chunks más similares a la consulta.
    Detalla los pasos del algoritmo para una mejor comprensión.
    """
    print("\n--- INICIO DEL PROCESO DE RECUPERACIÓN (RETRIEVAL) ---")
    print(f"Paso 1: Procesando la consulta original: '{query}'")

    # Paso 1.1: Tokenizar la consulta
    # Se convierte el texto de la consulta en una lista de palabras (tokens).
    query_tokens = tokenize(query)
    print(f"  -> Tokens de la consulta: {query_tokens}")

    # Paso 1.2: Convertir la consulta en un vector (embedding)
    # Se promedian los vectores de las palabras conocidas en el diccionario de embeddings.
    # Este vector representa el significado semántico de la consulta.
    query_vector = chunk_to_vector(query_tokens, embeddings_dict)
    print(f"  -> Vector (embedding) de la consulta: {query_vector}")
    if np.all(query_vector == 0):
        print("  -> AVISO: El vector de la consulta es un vector de ceros. Esto puede ocurrir si ninguna de las palabras de la consulta está en el diccionario de embeddings.")
        print("     La similitud coseno con un vector de ceros siempre será 0, lo que afectará los resultados de la recuperación.")
        print("     Considere añadir más palabras al 'embeddings_dict' o mejorar el preprocesamiento de la consulta para palabras fuera de vocabulario.")

    print("\nPaso 2: Calculando la similitud coseno con cada fragmento del corpus:")
    scores = []
    for idx, chunk_vec in enumerate(chunk_vectors):
        # Calcula la similitud coseno entre el vector de la consulta y cada vector de fragmento del corpus.
        # Una similitud cercana a 1.0 indica alta relación semántica.
        similarity = cosine_similarity(query_vector, chunk_vec)
        scores.append(similarity)
        # Se muestra la similitud calculada para cada fragmento.
        print(f"  -> Similitud con Doc ID '{chunks[idx]['doc_id']}' ('{chunks[idx]['fragmento']}'): {similarity:.4f}")
    print(f"  -> Puntuaciones de similitud de todos los fragmentos: {scores}")

    # Paso 3: Ordenar los fragmentos por similitud
    # Los fragmentos se clasifican de mayor a menor similitud con la consulta.
    print("\nPaso 3: Ordenando los fragmentos por similitud (de mayor a menor):")
    ranked = sorted(
        zip(range(len(chunks)), scores),
        key=lambda item: item[1],
        reverse=True
    )
    for rank_pos, (original_idx, score) in enumerate(ranked):
        print(f"  -> Rank #{rank_pos+1}: Doc ID '{chunks[original_idx]['doc_id']}' (Similitud: {score:.4f})")
    print(f"  -> Lista de fragmentos clasificados (índice original del chunk, puntuación de similitud): {ranked}")

    # Paso 4: Seleccionar los top_k fragmentos más relevantes
    # Se eligen los 'top_k' fragmentos con las puntuaciones de similitud más altas.
    print(f"\nPaso 4: Seleccionando los top {top_k} fragmentos más relevantes:")
    top_hits = []
    for rank_pos, (idx, score) in enumerate(ranked[:top_k]):
        chunk = chunks[idx]
        hit_info = {
            "rank": rank_pos + 1,
            "similitud": score,
            "fragmento": chunk["fragmento"],
            "doc_id": chunk["doc_id"],
        }
        top_hits.append(hit_info)
        print(f"  -> Seleccionado #{rank_pos+1} (Similitud: {score:.4f}): Doc ID '{chunk['doc_id']}' - '{chunk['fragmento']}'")

    print("\n--- FIN DEL PROCESO DE RECUPERACIÓN ---\n")
    return top_hits

def mostrar_resultados(pregunta, top_k=3):
    """Muestra los resultados de la búsqueda de forma legible y detallada."""
    print(f"\n==== MOSTRANDO RESULTADOS PARA LA CONSULTA: '{pregunta}' (Top {top_k} fragmentos) ====\n")
    resultados = retrieve(pregunta, top_k=top_k)

    print("\n===== RESULTADOS FINALES DE LA RECUPERACIÓN =====\n")
    if not resultados:
        print("No se encontraron resultados relevantes o el vector de la consulta fue nulo.")
        return

    for pos, hit in enumerate(resultados, start=1):
        porcentaje = round(hit["similitud"] * 100, 1)
        print(f"-------------------- RESULTADO RECUPERADO #{pos} --------------------")
        print(f"  Posición (Rank): {hit['rank']}")
        print(f"  Similitud con la consulta: {porcentaje}%")
        print(f"  Documento ID: {hit['doc_id']}")
        print(f"  Fragmento recuperado: '{hit['fragmento']}'")
        print("------------------------------------------------------------------\n")

# Ejemplo 1: Pregunta sobre mascotas
ejemplo1 = "¿Qué son las mascotas?"
mostrar_resultados(ejemplo1)



==== MOSTRANDO RESULTADOS PARA LA CONSULTA: '¿Qué son las mascotas?' (Top 3 fragmentos) ====


--- INICIO DEL PROCESO DE RECUPERACIÓN (RETRIEVAL) ---
Paso 1: Procesando la consulta original: '¿Qué son las mascotas?'
  -> Tokens de la consulta: ['¿qué', 'son', 'las', 'mascotas?']
  -> Vector (embedding) de la consulta: [0. 1. 0. 0. 0.]

Paso 2: Calculando la similitud coseno con cada fragmento del corpus:
  -> Similitud con Doc ID 'doc1' ('Los gatos son mascotas populares.'): 0.6871
  -> Similitud con Doc ID 'doc2' ('Los perros también son mascotas comunes.'): 0.7860
  -> Similitud con Doc ID 'doc3' ('La programación requiere práctica constante.'): 0.2128
  -> Puntuaciones de similitud de todos los fragmentos: [0.6870791209916813, 0.7860124955864106, 0.2127866687378167]

Paso 3: Ordenando los fragmentos por similitud (de mayor a menor):
  -> Rank #1: Doc ID 'doc2' (Similitud: 0.7860)
  -> Rank #2: Doc ID 'doc1' (Similitud: 0.6871)
  -> Rank #3: Doc ID 'doc3' (Similitud: 0.2128)
  -> Li

---

## Actividad práctica 💪

Implementa la función `analizar_similitud_palabras` para calcular la similitud coseno entre dos palabras.
Debe tomar dos palabras y el diccionario de embeddings, y devolver un valor flotante.
Considera:
   - Si alguna palabra no está en embeddings_dict, imprimir una advertencia y devolver 0.0.
   - Utilizar la función `cosine_similarity` ya definida.


In [29]:
import numpy as np

def analizar_similitud_palabras(palabra1: str, palabra2: str, embeddings_dict: dict) -> float:
    """
    Calcula la similitud coseno entre los vectores de dos palabras dadas.

    Esta función recupera los vectores (embeddings) de `palabra1` y `palabra2`
    del `embeddings_dict` y luego calcula su similitud coseno. Si alguna de las
    palabras no se encuentra en el diccionario, imprime una advertencia y devuelve 0.0.

    Args:
        palabra1 (str): La primera palabra para comparar.
        palabra2 (str): La segunda palabra para comparar.
        embeddings_dict (dict): Un diccionario donde las claves son palabras y los
                                valores son sus respectivos vectores (embeddings) NumPy.

    Returns:
        float: El valor de similitud coseno entre las dos palabras (entre -1 y 1).
               Devuelve 0.0 si alguna palabra no se encuentra en el `embeddings_dict`.
    """
    # --- TU CÓDIGO AQUÍ ---
    # Pista: Accede a los vectores de las palabras desde embeddings_dict
    #       Usa la función cosine_similarity
    #       Maneja el caso donde las palabras no existan en el diccionario.
    pass # Elimina esta línea cuando comiences a implementar la función

### Paso 2: Analiza los resultados

Observa las similitudes calculadas.
- ¿Por qué algunas preguntas recuperan mejor ciertos documentos?
- ¿Qué pasa cuando usas palabras que no están en el corpus?
- ¿Hay información semántica en los vectores?


In [32]:
# Prueba tu implementación cambiando las palabras

# Similitud entre 'mascotas' y 'perros'
palabra1 = "mascotas"
palabra2 = "perros"
print(f"La similitud entre {palabra1} y {palabra2} es {analizar_similitud_palabras(palabra1, palabra2, embeddings_dict)}")

print("\nRecuerda:\n- Una similitud cercana a 1 indica que las palabras tienen un significado similar en el espacio vectorial.")
print("- Una similitud cercana a 0 (o negativa) indica que son muy diferentes o no relacionadas.")
print("- Si una palabra no está en el 'embeddings_dict', la función indicará una advertencia y devolverá 0.0 de similitud.")

La similitud entre mascotas y perros es None

Recuerda:
- Una similitud cercana a 1 indica que las palabras tienen un significado similar en el espacio vectorial.
- Una similitud cercana a 0 (o negativa) indica que son muy diferentes o no relacionadas.
- Si una palabra no está en el 'embeddings_dict', la función indicará una advertencia y devolverá 0.0 de similitud.
